pip install paho-mqtt

!pip install pymysql

pip install --upgrade certifi

pip install pymysqlpool

pip install scikit-learn==1.3.1

In [ ]:
# # posture_hitory 더미데이터 만들기

# import pymysql
# from datetime import datetime, timedelta
# import random

# # MySQL 연결 설정
# mydb = pymysql.connect(
#     host='project-db-campus.smhrd.com',
#     port=3307,
#     user='ttap',
#     password='1234',
#     database='ttap',
#     cursorclass=pymysql.cursors.DictCursor
# )

# cursor = mydb.cursor()

# post_posture = 'DP'
# min_sum = 0

# # 시작 시간과 종료 시간의 범위 설정
# start_date = datetime(2023, 12, 15, 22, 0, 0)
# end_date = datetime(2024, 1, 10, 6, 0, 0)

# current_date = start_date
# begin_date = current_date
# stop_date = begin_date
# posture = random.choice(['DP', 'CP'])
# while current_date < end_date:
    

    
#     random_minutes = random.randint(30, 120)  # 랜덤한 분 설정 (30분 ~ 2시간)
#     end_minutes = random.randint(1, 9)  # 랜덤한 분 추가 (1분 ~ 9분)
    
#     min_sum += random_minutes
#     min_sum += end_minutes
    
#     if min_sum > 480:
#         min_sum = 480
    
#     stop_date = current_date + timedelta(minutes=min_sum)
    
#     # MySQL에 데이터 삽입
#     sql = "INSERT INTO posture_history (member_id, posture, start_time, end_time) VALUES (%s, %s, %s, %s)"
#     val = ('test', posture, begin_date.strftime("%Y-%m-%d %H:%M:%S"), stop_date.strftime("%Y-%m-%d %H:%M:%S"))
#     cursor.execute(sql, val)

#     if min_sum == 480:
#         min_sum = 0
#         current_date += timedelta(days=1)
#         posture = random.choice(['DP', 'CP'])
#         begin_date = current_date
#     else:
#         if posture == 'DP':
#             posture = 'CP'
#         else: 
#             posture ='DP'
#         begin_date = stop_date
        
# mydb.commit()


In [ ]:
import paho.mqtt.client as mqtt
import pymysql
import json
from datetime import datetime, timedelta
import time
import threading
import numpy as np
import joblib

# 쓰레드를 정지시킬 이벤트
stop_event = threading.Event()

# 머신러닝 모델 불러오기
# model_filename = "posture_model_knn.pkl"
# loaded_model = joblib.load(model_filename)
model_filename1 = "posture_model_knnrepair1.pkl"
loaded_model1 = joblib.load(model_filename1)
model_filename2 = "posture_model_knnf2.pkl"
loaded_model2 = joblib.load(model_filename2)
model_filename3 = "posture_model_knnf3.pkl"
loaded_model3 = joblib.load(model_filename3)
model_filename4 = "posture_model_knnf4.pkl"
loaded_model4 = joblib.load(model_filename4)
model_filename5 = "posture_model_knnf5.pkl"
loaded_model5 = joblib.load(model_filename5)

# MySQL 연결 설정
connection = pymysql.connect(
    host='project-db-campus.smhrd.com',
    port=3307,
    user='ttap',
    password='1234',
    database='ttap',
    cursorclass=pymysql.cursors.DictCursor
)

nowposture = None
nowlevel = None
dplevel = None
cplevel = None  
nowposture_temp = None
nowposture_temp_cnt = 0
cnt_check = True

# 사용자 설정 높이 조회
def check_height():
    print("check_height")
    global nowposture, nowlevel, dplevel, cplevel, nowposture_temp, nowposture_temp_cnt
    try:
        with connection.cursor() as cursor:
            sql = "SELECT cplevel, dplevel, nowposture, nowlevel FROM user_settings WHERE member_id = 'test1'"
            cursor.execute(sql)
            result = cursor.fetchone()
            if result:
                nowposture = result['nowposture']
                nowlevel = result['nowlevel']
                dplevel = result['dplevel']
                cplevel = result['cplevel']
                nowposture_temp = nowposture
                print(f"로드 완료 : 베개자세 : {nowposture} / 베개레벨 : {nowlevel} / DP레벨 : {dplevel} / CP레벨 : {cplevel} / 현재자세 : {nowposture_temp}")
    except pymysql.Error as e:
        print(f"Error checking height: {e}")

# 사용자 높이 변경
def change_height():
    print("change_height")
    global nowposture, nowlevel, dplevel, cplevel, nowposture_temp, nowposture_temp_cnt, cnt_check
    print(f"변경전 : 베개자세 : {nowposture} / 베개레벨 : {nowlevel} / DP레벨 : {dplevel} / CP레벨 : {cplevel}")
        
    air_check = ""
    air_level = 0

    if nowposture == "DP":
        if dplevel - nowlevel > 0:
            air_check = "in"
            air_level = dplevel - nowlevel
        elif dplevel - nowlevel < 0:
            air_check = "out"
            air_level = nowlevel - dplevel
        nowlevel = dplevel
    elif nowposture == "CP":
        if cplevel - nowlevel > 0:
            air_check = "in"
            air_level = cplevel - nowlevel
        elif cplevel - nowlevel < 0:
            air_check = "out"
            air_level = nowlevel - cplevel
        nowlevel = cplevel
    
    if air_level != 0:
        change_nowposlevel()
        cnt_check = False
        client.publish("Apill/height/return", json.dumps({"air_check": air_check, "air_level": air_level}))
        print(f"변경후 : 베개레벨 : {nowlevel}")
        
    level_check()

# 시용자 현재 자세 변경
def change_nowposlevel():
    print("change_nowposlevel")
    global nowposture, nowlevel, dplevel, cplevel, nowposture_temp, nowposture_temp_cnt
    try:
        # 데이터베이스 업데이트
        with connection.cursor() as cursor:
            sql = "UPDATE user_settings SET nowposture = %s, nowlevel = %s WHERE member_id = 'test1'"
            cursor.execute(sql, (nowposture, nowlevel,))
            connection.commit()
            
        print(f"베개자세 : {nowposture}, 베개레벨 : {nowlevel} 업데이트")

    except (json.JSONDecodeError, KeyError, pymysql.Error) as e:
        print(f"Error changing level: {e}")
        
# 사용자 설정 높이 변경 
def change_cplevel(payload):
    print("change_cplevel")
    global nowposture, nowlevel, dplevel, cplevel, nowposture_temp, nowposture_temp_cnt
    try:
        # 여기에서 필요한 유효성 검사 등을 수행
        if json.loads(payload)['index'] == 'change':
            new_cplevel = int(json.loads(payload)['level'])
        elif json.loads(payload)['index'] == 'plus':
            new_cplevel = cplevel + int(json.loads(payload)['level'])
        elif json.loads(payload)['index'] == 'minus':
            new_cplevel = cplevel - int(json.loads(payload)['level'])

        msg = 0
        
        if cplevel == new_cplevel:
            # msg = "현재 같은 높이입니다."
            msg = 1
        elif cplevel == 5 and new_cplevel > 5:
            # msg = "현재 최대 높이입니다."
            msg = 2
        elif cplevel == 1 and new_cplevel < 1:
            # msg = "현재 최소 높이입니다."
            msg = 3
        else:
            if new_cplevel > 5:
                # msg = "최대 높이인 5단계로 변경되었습니다."
                msg = 4
                new_cplevel = 5
            elif new_cplevel <= 5 and new_cplevel >= 1:
                # msg = str(new_cplevel) + "단계로 변경되었습니다."
                msg = 5
            elif new_cplevel < 1:
                # msg = "최소 높이인 1단계로 변경되었습니다."
                msg = 6
                new_cplevel = 1

            # 데이터베이스 업데이트
            with connection.cursor() as cursor:
                sql = "UPDATE user_settings SET cplevel = %s WHERE member_id = 'test1'"
                cursor.execute(sql, (new_cplevel,))
                connection.commit()
 
            cplevel = new_cplevel
            change_height()
            print(f"CP 레벨 업데이트: {cplevel}")
            
        client.publish("Apill/height/return/msg", json.dumps({"msg" : msg, "level" : str(cplevel)}, default=str))

    except (json.JSONDecodeError, KeyError, pymysql.Error) as e:
        print(f"Error changing CP level: {e}")

def change_dplevel(payload):
    print("change_dplevel")
    global nowposture, nowlevel, dplevel, cplevel, nowposture_temp, nowposture_temp_cnt
    try:
        # 여기에서 필요한 유효성 검사 등을 수행
        if json.loads(payload)['index'] == 'change':
            new_dplevel = int(json.loads(payload)['level'])
        elif json.loads(payload)['index'] == 'plus':
            new_dplevel = dplevel + int(json.loads(payload)['level'])
        elif json.loads(payload)['index'] == 'minus':
            new_dplevel = dplevel - int(json.loads(payload)['level'])
            
        msg = 0
        
        if dplevel == new_dplevel:
            # msg = "현재 같은 높이입니다."
            msg = 1
        elif dplevel == 5 and new_dplevel > 5:
            # msg = "현재 최대 높이입니다."
            msg = 2
        elif dplevel == 1 and new_dplevel < 1:
            # msg = "현재 최소 높이입니다."
            msg = 3
        else:
            if new_dplevel > 5:
                # msg = "최대 높이인 5단계로 변경되었습니다."
                msg = 4
                new_dplevel = 5
            elif new_dplevel <= 5 and new_dplevel >= 1:
                # msg = str(new_cplevel) + "단계로 변경되었습니다."
                msg = 5
            elif new_dplevel < 1:
                # msg = "최소 높이인 1단계로 변경되었습니다."
                msg = 6
                new_dplevel = 1
            
            # 데이터베이스 업데이트
            with connection.cursor() as cursor:
                sql = "UPDATE user_settings SET dplevel = %s WHERE member_id = 'test1'"
                cursor.execute(sql, (new_dplevel,))
                connection.commit()

            dplevel = new_dplevel
            change_height()
            print(f"DP 레벨 업데이트: {dplevel}")

        client.publish("Apill/height/return/msg", json.dumps({"msg" : msg, "level" : str(dplevel)}, default=str))

    except (json.JSONDecodeError, KeyError, pymysql.Error) as e:
        print(f"Error changing DP level: {e}")
        
def change_level(payload):
    print("change_level")
    global nowposture, nowlevel, dplevel, cplevel, nowposture_temp, nowposture_temp_cnt
    try:
        new_dplevel = int(json.loads(payload)['DP'])
        new_cplevel = int(json.loads(payload)['CP'])
        
        if new_dplevel == dplevel:
            pass
        else:
            # dp 레벨이 변경되었습니다.
            pass
        if new_cplevel == cplevel:
            pass
        else:
            # cp 레벨이 변경되었습니다.
            pass
        
        # 데이터베이스 업데이트
        with connection.cursor() as cursor:
            sql = "UPDATE user_settings SET dplevel = %s, cplevel = %s WHERE member_id = 'test1'"
            cursor.execute(sql, (new_dplevel, new_cplevel,))
            connection.commit()

        dplevel = new_dplevel
        cplevel = new_cplevel
        change_height()

#         client.publish("Apill/App/height/return", json.dumps({"msg" : msg, "level" : str(dplevel)}, default=str))

    except (json.JSONDecodeError, KeyError, pymysql.Error) as e:
        print(f"Error changing DP level: {e}")
        
def level_check():
    print("level_check")
    global nowposture, nowlevel, dplevel, cplevel, nowposture_temp, nowposture_temp_cnt
    try:
        client.publish("Apill/App/height/return", json.dumps({"nowposture" : nowposture, "nowlevel" : nowlevel, "dplevel" : dplevel, "cplevel" : cplevel}, default=str))
    except (json.JSONDecodeError, KeyError, pymysql.Error) as e:
        print(f"Error changing DP level: {e}")
        

log_posture = ""
log_starttime = None
# 자세 로그       
def record_posture_change(posture_type):
    try:
        global nowposture, log_posture, log_starttime

        if log_posture != "":
            log_endttime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            with connection.cursor() as cursor:
                sql = "UPDATE posture_history SET end_time = %s WHERE member_id = 'test1' AND start_time = %s AND end_time IS NULL"
                cursor.execute(sql, (log_endttime, log_starttime))
                connection.commit()
                
        # 자세가 변경되었을 때 시작 시간 기록
        if log_posture != nowposture:
            log_posture = nowposture
            log_starttime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            with connection.cursor() as cursor:
                sql = "INSERT INTO posture_history (member_id, posture, start_time) VALUES (%s, %s, %s)"
                cursor.execute(sql, ('test1', log_posture, log_starttime))
                connection.commit()    

                
    except Exception as e:
        print(f"자세 변경 기록 중 오류 발생: {e}")

        
def process_sensor_data(payload):
    global nowposture, nowlevel, dplevel, cplevel, nowposture_temp, nowposture_temp_cnt, cnt_check, sensor_check
    try:
        
        if sensor_check:
        
            # 문자열을 쉼표로 분할하고 양쪽 공백 제거
            sensor_values = [value.strip() for value in payload.replace('"', '').split(',') if value.strip()]

            if len(sensor_values) == 32:
                # 모든 값을 실수로 변환
                sensor_values = list(map(float, sensor_values))

                # NumPy 배열로 변환하고 reshape
                input_data = np.array(sensor_values).reshape(1, -1)

                # 예측
    #             predicted_label = loaded_model.predict(input_data)[0]

                if nowlevel == 1:
                    predicted_label = loaded_model1.predict(input_data)[0]
                elif nowlevel == 2:
                    predicted_label = loaded_model2.predict(input_data)[0]
                elif nowlevel == 3:
                    predicted_label = loaded_model3.predict(input_data)[0]
                elif nowlevel == 4:
                    predicted_label = loaded_model4.predict(input_data)[0]
                elif nowlevel == 5:
                    predicted_label = loaded_model5.predict(input_data)[0]

                # 현재 시간 가져오기
                current_time = datetime.now().strftime("%H:%M")

                print(f"현재 레벨 : {nowlevel} 베개자세 : {nowposture} 판단하는자세 : {predicted_label[-2:]} 자세한자세 : {predicted_label}")

                # 분류 결과 확인
                if predicted_label[-2:] == 'DP':
                    if nowposture_temp != 'DP':
                        nowposture_temp = 'DP'
                        nowposture_temp_cnt = 0
                    elif nowposture_temp == 'DP':
                        if cnt_check:
                            nowposture_temp_cnt += 1
                        if nowposture_temp_cnt == 4:
                            nowposture = 'DP'
                            change_height()
                            print("DP 높이 변경")
                            record_posture_change('DP')
                    print(f"현재 자세:{nowposture_temp} 현재 자세 카운트:{nowposture_temp_cnt}")

                elif predicted_label[-2:] == 'CP':
                    if nowposture_temp != 'CP':
                        nowposture_temp = 'CP'
                        nowposture_temp_cnt = 0
                    elif nowposture_temp == 'CP':
                        if cnt_check:
                            nowposture_temp_cnt += 1
                        if nowposture_temp_cnt == 4:
                            nowposture = 'CP'
                            change_height()
                            print("CP 높이 변경")
                            record_posture_change('CP')
                    print(f"현재 자세:{nowposture_temp} 현재 자세 카운트:{nowposture_temp_cnt}")

    except Exception as e:
        print(f"Error processing sensor data: {e}")

# 알람 목록을 저장할 리스트
alarms = []
alarm_sendtime = ''

# 알람 목록 조회 함수
def get_alarms():
    print("get_alarms")
    global alarms
    try:
        with connection.cursor() as cursor:
            # 알람 목록을 데이터베이스에서 조회
            sql = "SELECT id, time, isOn, isSelected FROM alarms"
            cursor.execute(sql)
            alarms = cursor.fetchall()

        # datetime 객체를 문자열로 변환 또는 None인 경우 그대로 둠
        for alarm in alarms:
            if isinstance(alarm['time'], datetime):
                alarm['time'] = alarm['time'].strftime("%H:%M")
                
        print(alarms)

    except pymysql.Error as e:
        print(f"Error getting alarms: {e}")
            
# 알람 추가 함수
def add_alarm(payload):
    print("add_alarm")
    try:
        alarm_data = json.loads(payload)
        with connection.cursor() as cursor:
            # 알람 정보를 데이터베이스에 추가
            sql = "INSERT INTO alarms (time, isOn, isSelected) VALUES (%s, %s, %s)"
            cursor.execute(sql, (alarm_data['time'], alarm_data['isOn'], alarm_data['isSelected']))
            connection.commit()
        get_alarms()
        print(f"알람 추가: {alarm_data}")
    except (json.JSONDecodeError, pymysql.Error) as e:
        print(f"Error adding alarm: {e}")

# 알람 수정 함수
def update_alarm(payload):
    print("update_alarm")
    try:
        alarm_data = json.loads(payload)
        with connection.cursor() as cursor:
            # 알람 정보를 데이터베이스에서 수정
            sql = "UPDATE alarms SET time=%s, isOn=%s, isSelected=%s WHERE id=%s"
            cursor.execute(sql, (alarm_data['time'], alarm_data['isOn'], alarm_data['isSelected'], alarm_data['id']))
            connection.commit()
        get_alarms()
        print(f"알람 업데이트: {alarm_data}")
    except (json.JSONDecodeError, pymysql.Error) as e:
        print(f"Error updating alarm: {e}")

# 알람 삭제 함수
def delete_alarm(payload):
    print("delete_alarm")
    try:
        alarm_id = json.loads(payload)['id']
        with connection.cursor() as cursor:
            # 알람 정보를 데이터베이스에서 삭제
            sql = "DELETE FROM alarms WHERE id=%s"
            cursor.execute(sql, (alarm_id,))
            connection.commit()
        get_alarms()
        print(f"알람 삭제: {alarm_id}")
    except (json.JSONDecodeError, pymysql.Error) as e:
        print(f"Error deleting alarm: {e}") 

# 알람 조회 함수
def send_alarm():
    print("send_alarm")
    try:
        global alarms
        client.publish("Apill/alarm/Appreturn", json.dumps(alarms, default=str))
    except pymysql.Error as e:
        print(f"Error getting alarms: {e}")
        
# 날씨 알람 함수
# def 
        
# SQL 문을 실행하고 결과를 MQTT로 전송하는 함수
def execute_sql_query(sql_query):
#     print("execute_sql_query")
    try:
        with connection.cursor() as cursor:
            # SQL 실행
            cursor.execute(sql_query)
            # 결과 가져오기
            result = cursor.fetchall() 
            # 결과를 MQTT 토픽에 전송
            client.publish("Apill/sql/return", json.dumps(result, default=str))
#         print(f"sql문 전송완료 {result}")
    except pymysql.Error as e:
        error_message = f"Error executing SQL: {e}"
        print(error_message)
        client.publish("Apill/sql/return", json.dumps(error_message, default=str))

# 회원가입
def insert_join(payload):
    try:
        # alarm_id = json.loads(payload)['id']
        member_data = json.loads(payload)
        
        with connection.cursor() as cursor:
            # SQL 실행
            sql = '''
                    INSERT INTO members (
                        member_id, member_pw, member_name, member_birth, 
                        member_weight, member_height, member_gender, member_age
                    ) VALUES (
                        %s, %s, %s, %s, %s, %s, %s, %s
                    )
                    '''
            print(bytes(member_data['name']).decode('utf-8'))
            cursor.execute(sql,(
                member_data['id'],
                member_data['pw'],
                bytes(member_data['name']).decode('utf-8'),
                member_data['birth'],
                member_data['weight'],
                member_data['height'],
                member_data['gender'],
                member_data['age']
            ))
            
            connection.commit()
            # 영향 받은 행의 수 확인
            affected_rows = cursor.rowcount
            
            # 결과를 MQTT 토픽에 전송
            json_result = json.dumps(affected_rows, default=str)
            print(f"SQL Query Result: {json_result}")
            client.publish("Apill/join/return", json_result)
    except pymysql.Error as e:
        error_message = f"Error executing SQL: {e}"
        print(error_message)
        client.publish("Apill/join/return", json.dumps(error_message, default=str))
        
# 회원가입 아이디 중복 검사
def join_id_check(payload):
    try:
        print(payload)
        user_id = json.loads(payload)['id']
        sql = 'select count(*) from members where member_id = %s'
        with connection.cursor() as cursor:
            # SQL 실행
            cursor.execute(sql,(user_id))
            # 결과 가져오기
            result = cursor.fetchall()
            # 결과를 MQTT 토픽에 전송
            json_result = json.dumps(result, default=str)
            print(f"SQL Query Result: {json_result}")
            client.publish("Apill/join/idcheck/return", json_result)
    except pymysql.Error as e:
        error_message = f"Error executing SQL: {e}"
        print(error_message)
        client.publish("Apill/join/idcheck/return", json.dumps(error_message, default=str))  
        
# 프로필 불러오기
def load_profile(payload):
    try:
        id_data = json.loads(payload)
        with connection.cursor() as cursor:
            # 알람 정보를 데이터베이스에 추가
            sql = "select * from members where id = %s"
            cursor.execute(sql, (id_data['id']))
            profile_data = cursor.fetchall()
            
            client.publish("Apill/user/profile/return", profile_data)
        print(f"id_profile: {id_data}")
    except (json.JSONDecodeError, pymysql.Error) as e:
        print(f"Error load_profile: {e}")

apill_check = False
apill_check_time = None
app_check = False
app_check_time = None

sensor_check = False
sensor_check_start = datetime.now()
sensor_check_end = datetime.now()

alarm_music_num = 1

# 주기적으로 체크하는 함수
def check_and_trigger():
    global alarms, alarm_sendtime, apill_check, app_check, sensor_check, sensor_check_start, sensor_check_end, apill_check_time, app_check_time, nowposture_temp_cnt, alarm_music_num
    while not stop_event.is_set():
        try:
            current_time = datetime.now()
#             apill_time_difference = 0
#             app_time_difference = 0
            # apill_check
            if apill_check:
                apill_check_time_datetime = datetime.strptime(apill_check_time, "%Y-%m-%d %H:%M:%S")
                apill_time_difference = current_time - apill_check_time_datetime
                if apill_time_difference.total_seconds() >= 25:
                    print("apill 종료")
                    apill_check = False
                else:
                    client.publish("Apill/RB/powercheck/return", json.dumps("powercheck", default=str))
                    
#             if app_check:
#                 app_check_time_datetime = datetime.strptime(app_check_time, "%Y-%m-%d %H:%M:%S")
#                 app_time_difference = current_time - app_check_time_datetime
#                 if app_time_difference.total_seconds() >= 25:
#                     print("app 종료")
#                     app_check = False
#                 else:
#                     client.publish("Apill/App/powercheck/return")
              
            alarm_weather_check = False
#             alarm_weather_difference = 0
            
            # 현재 시간을 확인하고 알람이 발생해야 하는지 여부를 판단
            alarm_current_time = datetime.now().strftime("%H:%M")  # 초 부분을 0으로 만듦
            for alarm in alarms:
                if alarm_sendtime != alarm_current_time:
                    alarm_time = alarm['time']
                    if alarm_time == alarm_current_time and alarm['isOn']:
                        print(f"알람발생! {alarm['time']}")
                        # / {alarm['index']}
                        client.publish("Apill/alarm/RBreturn", json.dumps({"msg" : alarm_music_num}, default=str))
                        if alarm_music_num == 5:
                            alarm_music_num = 1
                        else:
                            alarm_music_num += 1
                        alarm_sendtime = alarm_time
#                 alarm_weather_difference = abs(alarm_current_time - datetime.strptime(alarm_time, "%H:%M"))
#                 print(alarm_weather_difference)
                alarm_weather_time = datetime.strptime(alarm_time, "%H:%M")
                alarm_weather_start = current_time - timedelta(minutes=10)
                alarm_weather_end = current_time + timedelta(minutes=10)
                if alarm_weather_start.time() <= alarm_weather_time.time() <= alarm_weather_end.time():
                    alarm_weather_check = True
                else:
                    alarm_weather_check = False
                    
            # 센서값 들어온 시간 체크
            sensor_time_difference = sensor_check_end - sensor_check_start
            if sensor_check:
                if (current_time - sensor_check_end).total_seconds() >= 10:
                    print("sensor 종료")
                    sensor_time_difference = sensor_check_end - sensor_check_start
                    sensor_check = False
                    sensor_check_start = datetime.now()
                    sensor_check_end = datetime.now()
                    nowposture_temp_cnt = 0
                    # 날씨 알람 발송
                    if alarm_weather_check == True and sensor_check == False and sensor_time_difference.total_seconds() >= 10:
                        print("날씨 알람 발송 확인")
                        client.publish("Apill/RB/weather/return", json.dumps("alarmend", default=str))
                    

            
            time.sleep(5)  # 5초
        except Exception as e:
            print(f"Error in periodically: {e}")

# MQTT 브로커 설정
broker_address = "192.168.148.146"
broker_port = 1883

# MQTT 클라이언트 생성
client = mqtt.Client()

# MQTT 연결 시 호출되는 콜백 함수
def on_connect(client, userdata, flags, rc):
#     print(f"Connected with result code {rc}")
    
    # 토픽 구독
    # 온오프 체크
    client.subscribe("Apill/RB/powercheck")
    client.subscribe("Apill/App/powercheck")
    
    # 센서값
    client.subscribe("Apill/RB/sensor")
#     client.subscribe("Apill/sleepposture")

    # 알람
    client.subscribe("Apill/alarm/add")
    client.subscribe("Apill/alarm/update")
    client.subscribe("Apill/alarm/delete")
    client.subscribe("Apill/alarm/get")
    client.subscribe("Apill/alarm/check")
    
    # 높이변경
    client.subscribe("Apill/height")
    
    # 높이완료
    client.subscribe("Apill/height/complete")
    
    # App 높이 변경
    client.subscribe("Apill/App/height")
    
    # sql문 처리
    client.subscribe("Apill/sql")
    
    # 회원가입
    client.subscribe('Apill/join')
    client.subscribe('Apill/join/idcheck')
    
    # 프로필
    client.subscribe('Apill/user/profile')
    
# MQTT 메시지 수신 시 호출되는 콜백 함수
def on_message(client, userdata, msg):
    global apill_check, apill_check_time, app_check, app_check_time, sensor_check, sensor_check_start, sensor_check_end, cnt_check

#     print(f"받은 메세지: {msg.payload.decode('utf-8')}")

    # 센서값 처리
    if msg.topic == "Apill/RB/sensor":
        if sensor_check == False:
            print("sensor 시작")
            sensor_check = True
            sensor_check_start = datetime.now()
        sensor_check_end = datetime.now()
            
        process_sensor_data(msg.payload.decode('utf-8'))
        
    # 전원 체크
    elif msg.topic == "Apill/RB/powercheck":
        if apill_check == False:
            print("apill 시작")
            apill_check = True
        apill_check_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#         print(f"Apill/RB/powercheck {apill_check_time}")
    elif msg.topic == "Apill/App/powercheck":
        if app_check == False:
            print("app 시작")
#             level_check()
            app_check = True
        level_check()
        app_check_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        client.publish("Apill/App/powercheck/return")
#         print(f"Apill/App/powercheck {app_check_time}")

    # 알람 목록 조회 메시지 처리
    elif msg.topic == "Apill/alarm/get":
        get_alarms()
    elif msg.topic == "Apill/alarm/check":
        send_alarm()
    # 알람 추가 메시지 처리
    elif msg.topic == "Apill/alarm/add":
        add_alarm(msg.payload.decode('utf-8'))
        send_alarm()
    # 알람 수정 메시지 처리
    elif msg.topic == "Apill/alarm/update":
        update_alarm(msg.payload.decode('utf-8'))
        send_alarm()
    # 알람 삭제 메시지 처리
    elif msg.topic == "Apill/alarm/delete":
        delete_alarm(msg.payload.decode('utf-8'))
        send_alarm()
        
    # 높이
    elif msg.topic == "Apill/height":
        if nowposture == "DP":
            change_dplevel(msg.payload.decode('utf-8'))
        elif nowposture == "CP":
            change_cplevel(msg.payload.decode('utf-8'))
            
    # 높이 완료
    elif msg.topic == "Apill/height/complete":
        print("높이변경완료")
        cnt_check = True
            
    # 앱 높이
    elif msg.topic == "Apill/App/height":
        
        change_level(msg.payload.decode('utf-8'))

    # SQL 문을 받아서 실행하고 결과를 MQTT로 전송
    elif msg.topic == "Apill/sql":
        execute_sql_query(msg.payload.decode('utf-8'))
        
    # 회원가입
    elif msg.topic == "Apill/join":
        insert_join(msg.payload.decode('utf-8'))
    elif msg.topic == "Apill/join/idcheck":
        join_id_check(msg.payload.decode('utf-8'))

# MQTT 클라이언트에 콜백 함수 등록
client.on_connect = on_connect
client.on_message = on_message

# MQTT 브로커에 연결
client.connect(broker_address, broker_port, 60)

check_height()
change_height()
get_alarms()

# 시작
# 주기적으로 확인하는 쓰레드 시작
alarm_thread = threading.Thread(target=check_and_trigger)
alarm_thread.start()

# 브로커 서버 루프 시작
try:
    client.loop_forever()
except KeyboardInterrupt:
    print("KeyboardInterrupt: Stopping threads and disconnecting MQTT.")
    stop_event.set()  # 쓰레드 종료를 위해 이벤트 설정
    alarm_thread.join()  # 쓰레드 종료 대기
    client.disconnect()  # MQTT 연결 종료
except Exception as e:
    print(f"Unhandled exception: {e}")

print("Exiting program.")

check_height
로드 완료 : 베개자세 : DP / 베개레벨 : 1 / DP레벨 : 1 / CP레벨 : 3 / 현재자세 : DP
change_height
변경전 : 베개자세 : DP / 베개레벨 : 1 / DP레벨 : 1 / CP레벨 : 3
level_check
get_alarms
[{'id': 44, 'time': '13:29', 'isOn': 1, 'isSelected': 0}]
apill 시작
sensor 시작
현재 레벨 : 1 베개자세 : DP 판단하는자세 : CP 자세한자세 : CP
현재 자세:CP 현재 자세 카운트:0


C:\Users\lee\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


sensor 종료
app 시작
level_check
sensor 시작
현재 레벨 : 1 베개자세 : DP 판단하는자세 : DP 자세한자세 : DP
현재 자세:DP 현재 자세 카운트:0


C:\Users\lee\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\lee\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


현재 레벨 : 1 베개자세 : DP 판단하는자세 : DP 자세한자세 : DP
현재 자세:DP 현재 자세 카운트:1


...

현재 레벨 : 3 베개자세 : CP 판단하는자세 : CP 자세한자세 : CP
현재 자세:CP 현재 자세 카운트:16


C:\Users\lee\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


현재 레벨 : 3 베개자세 : CP 판단하는자세 : CP 자세한자세 : CP
현재 자세:CP 현재 자세 카운트:16


C:\Users\lee\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


현재 레벨 : 3 베개자세 : CP 판단하는자세 : CP 자세한자세 : CP
현재 자세:CP 현재 자세 카운트:16
높이변경완료
sensor 종료
날씨 알람 발송 확인
알람발생! 14:32
apill 종료
